In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk.corpus
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import datasets, linear_model
import scipy.sparse as sps
from scipy.sparse import coo_matrix, hstack, vstack, csr_matrix
from scipy import io
from datetime import datetime
import gc
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
from IPython.display import display, HTML
from pprint import pprint

In [2]:
class FeatureEngineering:
    #ValidationStart = 0
    #lag = 0
    #trainHdfPath = ""
    #trainHdfFile = ""
    #testHdfPath = ""
    #testHdfFile = ""
    #train = pd.DataFrame()
    #test = pd.DataFrame()
    
    def __init__(self, ValidationStart, ValidationSmallerThan, trainHdfPath, trainHdfFile, testHdfPath, testHdfFile, 
                 testTypes, trainTypes, trainCsvPath, testCsvPath, maxLag=0):
        self.ValidationStart = ValidationStart
        self.ValidationSmallerThan = ValidationSmallerThan
        self.maxLag = maxLag
        self.trainHdfPath = trainHdfPath
        self.trainHdfFile = trainHdfFile
        self.testHdfPath = testHdfPath
        self.testHdfFile = testHdfFile
        self.testTypes = testTypes
        self.trainTypes = trainTypes
        self.trainCsvPath = trainCsvPath
        self.testCsvPath = testCsvPath
        
    @staticmethod
    def __printDataFrameBasics__(data):
        display(data.head(2))
        #print data.dtypes
        gc.collect()
        print(data.info(memory_usage=True))
        
    @staticmethod    
    def __changeIndexTypeToLowerMemory(data):
        ##########
        #This is very critical, i accept max number is 2^32. Also, if don't do that, memory gets so much higher..
        ##########
        data.index = data.index.astype('uint32')
        
    def ReadHdf(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"
            
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_hdf(self.trainHdfPath,self.trainHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_hdf(self.testHdfPath,self.testHdfFile)
            FeatureEngineering.__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)
        
    def ReadCsv(self, trainOrTestOrBoth):
        '''Reads and holds Df in object memory'''
        if trainOrTestOrBoth == '':
            raise "qweqweq"  
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train = pd.read_csv(self.trainCsvPath, usecols=self.trainTypes.keys(), dtype=self.trainTypes)
            #__changeIndexTypeToLowerMemory(self.train)
            FeatureEngineering.__printDataFrameBasics__(self.train)
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test = pd.read_csv(self.testCsvPath, usecols=self.testTypes.keys(), dtype=self.testTypes)
            #__changeIndexTypeToLowerMemory(self.test)
            FeatureEngineering.__printDataFrameBasics__(self.test)

    @staticmethod
    def ConvertCsvToHdf(csvPath, HdfPath, HdfName, ColumnTypeDict ):
        tempDf = pd.read_csv(csvPath, usecols=ColumnTypeDict.keys(), dtype=ColumnTypeDict)
        tempDf.to_hdf(HdfPath, HdfName, format='t')
        print "ConvertCsvToHdf is done.."

    def Preprocess(self, trainOrTestOrBoth, columnFunctionTypeList):
        '''columnFunctionTypeList = [ ['C1',Func1,Type], ['C2',Func2,Type],..    ]'''
        for column, func, localType in columnFunctionTypeList:
            if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
                self.train.loc[:,column] =  self.train[column].apply(func).astype(localType)
            elif trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
                self.test.loc[:,column] =  self.test[column].apply(func).astype(localType)
            
    def SaveDataFrameToHdf(self,trainOrTestOrBoth):
        if trainOrTestOrBoth == 'train' or trainOrTestOrBoth=='both':
            self.train.to_hdf(self.trainHdfPath, self.trainHdfFile, format='t')
        if trainOrTestOrBoth == 'test' or trainOrTestOrBoth=='both':
            self.test.to_hdf(self.testHdfPath, self.testHdfFile, format='t')
        
    def AddDemandaGeneralMean(self):
        DemandaMeanWithoutLag = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart]
        self.train.loc[:,"DemandaGeneralMean"] = DemandaMeanWithoutLag['Demanda_uni_equil'].mean()
        self.train.loc[:,"DemandaGeneralMean"] = self.train.loc[:,"DemandaGeneralMean"].astype('float32')
        display(self.train)
        
    def AddConfigurableFeaturesToTrain(self, config):
        if config.lag > self.maxLag:
            self.maxLag = config.lag
        
        lag = 0 if config.lag == 0 else config.lag-1
        tempData = self.train.loc[self.train.loc[:,'Semana'] < self.ValidationStart - lag]
        #display(tempData)
        if(config.lag != 0):
            tempData.loc[:,'Semana'] = tempData.loc[:,'Semana'].apply(lambda x:x + config.lag)
        #display(tempData)
        
        #Means iterative.. eliminate as long as np.nan exists..If there is already one, don't create, use the existing
        if config.targetVariable != "" and  config.targetVariable not in self.train.columns:
            self.train.loc[:,config.targetVariable] = np.nan
        
        for name,groups in config.nameAndGroups:
            if name not in self.train.columns:
                print "{} is not in columns..".format(name)
                
                ##First, select columns not to fill memory..then groupby..
                
                groupedDataframe =  pd.DataFrame({name:tempData[groups+['Demanda_uni_equil']].groupby(groups)
                    ['Demanda_uni_equil'].mean().astype('float32')})
                #join..
                
                display(groupedDataframe)
                self.train = self.train.merge( groupedDataframe, left_on=groups,
                                  right_index=True, how='left', sort=False,copy=False)
            else:
                print "{} is in columns..".format(name)
            
            display(self.train)
            
            #Means iterative..!!!!!
            if config.targetVariable != "":
                self.train.loc[pd.isnull(self.train[config.targetVariable]), 
                    config.targetVariable] = self.train.loc[pd.isnull(self.train[config.targetVariable]), name]
                count = self.train.loc[self.train['Semana'] >= self.ValidationStart,config.targetVariable].isnull().sum()
                print "Count of missing numbers after {} in validation part in column {} is {}".format(name, 
                    config.targetVariable,str(count))
                #display(self.train)
                #If column is already in Dataframe and we want to fill target variable, this deletes columns!!!
                if(config.deleteColumns):
                    self.train.drop(name, axis=1, inplace=True)
                gc.collect()
                if count == 0:
                    break
        display(self.train)        
        gc.collect()
        return 
    def DeleteLaggedWeeksFromTrain(self):
        self.train = self.train.loc[self.train.loc[:,'Semana']>= 3 + self.maxLag]
        display(self.train)

In [3]:
parameterDict =       {"ValidationStart":6, 
 "ValidationSmallerThan":7,
   "maxLag":2,
    "trainHdfPath":'../../input/train.h5',
    "trainHdfFile":"train",
    "testHdfPath":"../../input/test.h5",
    "testHdfFile":"test", 
    "trainTypes" : {'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16, 
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16,'Venta_uni_hoy':np.uint16, 'Venta_hoy':np.float32,
                    'Dev_uni_proxima': np.uint32, 'Dev_proxima':np.float32,'Demanda_uni_equil':np.uint32}, 
    "testTypes" : {'id':np.uint32,'Semana':np.uint8, 'Agencia_ID':np.uint16, 'Canal_ID':np.uint8,'Ruta_SAK':np.uint16,
        'Cliente_ID':np.uint32, 'Producto_ID':np.uint16},
    "trainCsvPath":'../../input/train.csv'   ,
    "testCsvPath":'../../input/test.csv'}

FE = FeatureEngineering(**parameterDict)
print FE.__dict__

{'maxLag': 2, 'testTypes': {'Cliente_ID': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Semana': <type 'numpy.uint8'>, 'id': <type 'numpy.uint32'>}, 'testHdfFile': 'test', 'trainTypes': {'Dev_proxima': <type 'numpy.float32'>, 'Venta_uni_hoy': <type 'numpy.uint16'>, 'Cliente_ID': <type 'numpy.uint32'>, 'Demanda_uni_equil': <type 'numpy.uint32'>, 'Ruta_SAK': <type 'numpy.uint16'>, 'Canal_ID': <type 'numpy.uint8'>, 'Venta_hoy': <type 'numpy.float32'>, 'Producto_ID': <type 'numpy.uint16'>, 'Agencia_ID': <type 'numpy.uint16'>, 'Dev_uni_proxima': <type 'numpy.uint32'>, 'Semana': <type 'numpy.uint8'>}, 'trainCsvPath': '../../input/train.csv', 'testHdfPath': '../../input/test.h5', 'testCsvPath': '../../input/test.csv', 'ValidationStart': 6, 'trainHdfFile': 'train', 'trainHdfPath': '../../input/train.h5', 'ValidationSmallerThan': 7}


In [4]:
FE.ReadHdf('train')
FE.train.loc[:,'log_Demanda_uni_equil'] =  FE.train.loc[:,'Demanda_uni_equil'].apply(lambda x:round(np.expm1(x))).astype('uint32')
FE.train.drop("Demanda_uni_equil",axis=1,inplace=True)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,1.386294
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,1.609438


<class 'pandas.core.frame.DataFrame'>
Int64Index: 74180464 entries, 0 to 74180463
Data columns (total 11 columns):
Semana               uint8
Agencia_ID           uint16
Canal_ID             uint8
Ruta_SAK             uint16
Cliente_ID           uint32
Producto_ID          uint16
Venta_uni_hoy        uint16
Venta_hoy            float32
Dev_uni_proxima      uint32
Dev_proxima          float32
Demanda_uni_equil    float32
dtypes: float32(3), uint16(4), uint32(2), uint8(2)
memory usage: 2.3 GB
None


In [5]:
FE.train.count()

Semana                   74180464
Agencia_ID               74180464
Canal_ID                 74180464
Ruta_SAK                 74180464
Cliente_ID               74180464
Producto_ID              74180464
Venta_uni_hoy            74180464
Venta_hoy                74180464
Dev_uni_proxima          74180464
Dev_proxima              74180464
log_Demanda_uni_equil    74180464
dtype: int64

In [6]:
FE.test = FE.train[ FE.train.Semana>=8 ]

In [7]:
FE.train = FE.train[ FE.train.Semana<8 ]

In [8]:
FE.test[0:2]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
53364883,8,1110,7,3301,15766,1212,4,33.520000,0,0,4
53364884,8,1110,7,3301,15766,1216,5,41.900002,0,0,5


In [9]:
FE.train[0:2]

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,log_Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0,4


# Train include Test datas?

## Producto_ID Differences

In [10]:
testP = set(FE.test.Producto_ID)
trainP = set(FE.train.Producto_ID)

In [11]:
fileP = set(pd.read_csv("../../input/producto_tabla.csv").Producto_ID)

In [12]:
print len(testP)
print len(trainP)
print len(fileP)

1634
1737
2592


In [13]:
print trainP - fileP
print testP - fileP
print len(testP - trainP)
print testP - trainP

set([])
set([])
62
set([47123, 35713, 31618, 36995, 43398, 37639, 43193, 36873, 48138, 2060, 2062, 2064, 37011, 48023, 48232, 37402, 40987, 30595, 37149, 36131, 35240, 48297, 37469, 30877, 35761, 37363, 3509, 33208, 37667, 37690, 36671, 37403, 33735, 37576, 4169, 37578, 32207, 42192, 37587, 37590, 37689, 36699, 37468, 37242, 37470, 37264, 40929, 35301, 35304, 34665, 35306, 35308, 35310, 1277, 37379, 35441, 35442, 37139, 3446, 37241, 122, 43389])


## Cliente_ID Differences

In [14]:
testCl = set(FE.test.Cliente_ID)
trainCl = set(FE.train.Cliente_ID)
fileCl = set(pd.read_csv("../../input/cliente_tabla.csv").Cliente_ID)

In [15]:
print trainCl - fileCl
print testCl - fileCl
print len(testCl - trainCl)
#print testCl - trainCl


set([])
set([])
18337


## Agencia_ID Differences

In [16]:
testA = set(FE.test.Agencia_ID)
trainA = set(FE.train.Agencia_ID)
fileA = set(pd.read_csv("../../input/town_state.csv").Agencia_ID)

In [17]:
print trainA - fileA
print testA - fileA
print testA - trainA

set([])
set([])
set([])


## Ruta_SAK Differences

In [18]:
testR = set(FE.test.Ruta_SAK)
trainR = set(FE.train.Ruta_SAK)

In [19]:
print len(testR - trainR)
print testR - trainR
print len(trainR - testR)

108
set([8193, 8196, 9228, 8205, 3096, 9757, 7198, 9759, 3108, 9774, 9266, 7737, 7743, 9286, 1099, 9229, 7762, 8278, 8633, 9816, 7777, 7780, 7270, 8295, 7271, 8813, 7280, 8041, 8843, 9871, 7314, 8131, 9877, 8860, 8867, 9383, 9386, 7855, 7368, 2226, 7861, 9928, 8910, 8913, 7382, 7387, 7901, 7397, 9958, 8937, 7402, 8429, 8950, 7935, 3968, 7938, 8463, 9504, 9435, 7478, 7480, 9532, 9534, 9023, 7493, 8518, 8520, 7497, 9548, 9552, 9554, 8508, 9582, 6004, 6005, 6008, 3962, 8059, 3964, 3965, 3967, 9600, 3969, 8173, 8087, 8088, 8091, 4001, 4003, 4004, 8103, 9129, 4012, 4013, 9657, 8126, 8643, 9157, 7113, 4558, 8146, 8170, 9810, 8691, 9206, 9722, 7338, 9727])
596


In [20]:
testCh = set(FE.test.Canal_ID)
trainCh = set(FE.train.Canal_ID)

In [21]:
print testCh - trainCh
print trainCh - testCh

set([])
set([])


In [ ]:
## Producto_ID Cliente_ID

In [ ]:
testPC = pd.unique(FE.test[['Producto_ID', 'Cliente_ID']].values)

In [23]:
liste = [1,412,32,12,567,12451,23,1,0]

In [34]:
liste32 =  np.log1p(liste).astype("float32")
print np.mean(liste32)
print np.expm1(liste32)
print np.array([ round(i) for i in   np.expm1(liste32)]) - liste


3.60233
[  1.00000000e+00   4.11999969e+02   3.20000038e+01   1.19999990e+01
   5.67000122e+02   1.24510049e+04   2.30000000e+01   1.00000000e+00
   0.00000000e+00]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [33]:
liste32 =  np.log1p(liste).astype("float16")
print np.mean(liste32)
print np.expm1(liste32)
print np.array([ round(i) for i in   np.expm1(liste32)]) - liste


3.6035
[  1.00000000e+00   4.12000000e+02   3.19843750e+01   1.19921875e+01
   5.68000000e+02   1.24480000e+04   2.30000000e+01   1.00000000e+00
   0.00000000e+00]
[ 0.  0.  0.  0.  1. -3.  0.  0.  0.]


In [31]:
round(np.expm1(liste32)) 

TypeError: only length-1 arrays can be converted to Python scalars